# SQLite demo (chinook database)

This is a little demo of using SQL and Python to query a simple SQLite database. The example database being used here is called "chinook" and was taken from https://www.sqlitetutorial.net/sqlite-sample-database/, which contains information about an iTunes-like commerce site.

In [1]:
import sqlite3
import pandas as pd

# source: https://www.sqlitetutorial.net/sqlite-sample-database/
conn = sqlite3.connect('chinook.db')

In [2]:
# helper function to run queries in SQL files
def run_query_from_file(filename):
    with open(filename, 'r') as file:
        queries = file.read()
    # if there are multiple queries in the file, run the first one
    queries = queries.split(';')
    return pd.read_sql_query(queries[0], conn)

## Customer sales by support rep

I chose to include the people who don't have the role "Sales Support Agent" because I think doing so is a good confirmation that no one else does any support work.

In [3]:
run_query_from_file('sales_by_emp.sql').sort_values('SalesTotal', ascending=False)

,EmployeeId,FirstName,LastName,Title,CustomersSupported,SalesTotal
2,3,Jane,Peacock,Sales Support Agent,21,833.04
3,4,Margaret,Park,Sales Support Agent,20,775.40
4,5,Steve,Johnson,Sales Support Agent,18,720.16
0,1,Andrew,Adams,General Manager,0,0.00
1,2,Nancy,Edwards,Sales Manager,0,0.00
5,6,Michael,Mitchell,IT Manager,0,0.00
6,7,Robert,King,IT Staff,0,0.00
7,8,Laura,Callahan,IT Staff,0,0.00


## Top 10 artists by revenue

In [4]:
run_query_from_file('sales_by_artist.sql').head(10)

,Artist,Quantity,Revenue
0,Iron Maiden,140,138.60
1,U2,107,105.93
2,Metallica,91,90.09
3,Led Zeppelin,87,86.13
4,Lost,41,81.59
5,The Office,25,49.75
6,Os Paralamas Do Sucesso,45,44.55
7,Deep Purple,44,43.56
8,Faith No More,42,41.58
9,Eric Clapton,40,39.60


## Sales by country

In [22]:
run_query_from_file('sales_by_country.sql')

DatabaseError: Execution failed on sql 'select * from invoice_items ii
left join customers c on ii.customerid = c.customerid
where country = 'USA'': no such column: ii.customerid

## Sales by genre

In [6]:
run_query_from_file('sales_by_genre.sql')

,Genre,Quantity,Revenue
0,Rock,835,826.65
1,Latin,386,382.14
2,Metal,264,261.36
3,Alternative & Punk,244,241.56
4,TV Shows,47,93.53
5,Jazz,80,79.20
6,Blues,61,60.39
7,Drama,29,57.71
8,R&B/Soul,41,40.59
9,Classical,41,40.59


## Top 25 albums by revenue

In [7]:
# top 25 albums
# now let's do this using pandas instead of SQL

# pull all the tables we need into dataframes
df_inv_itms = pd.read_sql_query('select * from invoice_items', conn)
df_tracks = pd.read_sql_query('select * from tracks', conn)
df_albums = pd.read_sql_query('select * from albums', conn)
df_artists = pd.read_sql_query('select * from artists', conn)

# merge track and album info
df_sales_by_album = df_tracks.merge(df_albums, how='left', on='AlbumId')

# merge invoice items with track/album info
df_sales_by_album = df_inv_itms.merge(df_sales_by_album, how='left', on='TrackId')

# merge on artist info
df_sales_by_album = df_sales_by_album.merge(df_artists, how='left', on='ArtistId')

# clean it up a bit
df_sales_by_album = df_sales_by_album[['InvoiceLineId', 'UnitPrice_x', 'Quantity', 'Name_x', 'Title', 'Name_y']]
df_sales_by_album = df_sales_by_album.rename(columns={'UnitPrice_x':'UnitPrice',
                                    'Name_x': 'TrackName',
                                    'Title':'AlbumName',
                                    'Name_y': 'ArtistName'
                                   })
df_sales_by_album['Revenue'] = df_sales_by_album['Quantity'] * df_sales_by_album['UnitPrice']

# sum quantity and revenue by album/artist
df_sales_by_album = df_sales_by_album.groupby(
    ['AlbumName', 'ArtistName']
    )[['Quantity', 'Revenue']].sum().sort_values('Revenue', ascending=False).reset_index()
df_sales_by_album = df_sales_by_album.rename(columns={'Quantity':'QuantityTracksPurchased'})

df_sales_by_album.head(25)

,AlbumName,ArtistName,QuantityTracksPurchased,Revenue
0,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),18,35.82
1,"The Office, Season 3",The Office,16,31.84
2,Minha Historia,Chico Buarque,27,26.73
3,"Heroes, Season 1",Heroes,13,25.87
4,"Lost, Season 2",Lost,13,25.87
5,Greatest Hits,Lenny Kravitz,26,25.74
6,Unplugged,Eric Clapton,25,24.75
7,"Battlestar Galactica, Season 3",Battlestar Galactica,12,23.88
8,"Lost, Season 3",Lost,11,21.89
9,Acústico,Titãs,22,21.78


In [8]:
# conn.close()